In [58]:
import pandas as pd #data handling library
import sklearn as sk

def read_file_list(file_list_name):
    """

    :param file_list_name: name of file that containts list of feature file names (should be full paths)
    :return: list of files names
    """
    files = []
    with open(file_list_name, 'r') as file_list:
        for line in file_list:
            files.append(line)
    return files

#file_list = read_file_list("index_file.txt")

     ?|100130426 ?|100133144 ?|100134869  ?|10357  ?|10431 ?|136542 ?|155060  \
RPKM           0    0.809971    0.179658  3.67724  36.2023        0   3.7828   

       ?|26823 ?|280660 ?|317712     ...     ZXDA|7789 ZXDB|158586 ZXDC|79364  \
RPKM  0.197409        0        0     ...      0.686332     3.34044    10.9434   

     ZYG11A|440590 ZYG11B|79699 ZYX|7791 ZZEF1|23140 ZZZ3|26009  \
RPKM      0.614655      3.32428  51.6698     12.7978    7.19757   

     psiTPTE22|387590 tAKR|389932  
RPKM          14.0532    0.160082  

[1 rows x 20532 columns]


In [59]:
def get_RPKM_by_file(files):
    
    data_fields = ('gene','RPKM')
    RPKM_list = []
    for file_name in files:
        df = pd.read_table(filename, usecols=data_fields)
        df = df.T # tanspose
        new_header = df.iloc[0] #grab the first row for the header
        df = df[1:] #take the data less the header row
        df = df.rename(columns = new_header.T) #set the header row as the df header
        RPKM_list.append(df)
    return RPKM_list = []

flattened_data = get_RPKM_by_file(file_list)




     ?|100130426 ?|100133144 ?|100134869  ?|10357  ?|10431 ?|136542 ?|155060  \
RPKM           0    0.809971    0.179658  3.67724  36.2023        0   3.7828   

       ?|26823 ?|280660 ?|317712     ...     ZXDA|7789 ZXDB|158586 ZXDC|79364  \
RPKM  0.197409        0        0     ...      0.686332     3.34044    10.9434   

     ZYG11A|440590 ZYG11B|79699 ZYX|7791 ZZEF1|23140 ZZZ3|26009  \
RPKM      0.614655      3.32428  51.6698     12.7978    7.19757   

     psiTPTE22|387590 tAKR|389932  
RPKM          14.0532    0.160082  

[1 rows x 20532 columns]


In [ ]:
def merge_data(data_list):
    df = pd.concat(l, axis=0, join='inner', join_axes=None, ignore_index=False,
        keys=None, levels=None, names=None, verify_integrity=False)
    return df








In [ ]:
#machine learning

